In [13]:
# -*- coding: utf-8 -*-
import BPNN
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import scipy

def translate_output(arr):
    res = []
    for i in arr:
        max = 0
        id = 0
        mid = 0
        for num in i:
            if num > max:
                max = num
                mid = id
            id += 1
        res.append([mid])
    return np.array(res)
        
rp_list = []
def train_predict_stocks(dir, file_name, train_case_num):

    df = pd.read_csv(dir + file_name)
    train_case_num = int(len(df)*0.75)

    bpinput = df.loc[:,['mei', 'yesterday_p_change']].as_matrix()

    close_price = df.loc[:, ['close']].as_matrix().tolist()
    p_change = df.loc[:, ['p_change']].as_matrix().tolist()
    close_price = [i[0] for i in close_price]
    p_change = [i[0] for i in p_change]
    pre_close_price = close_price.copy()

    bpoutput = df['normalized_p_change']
    output_vec = []
    for row in bpoutput:
        row_vec = [0] * 10
        row_vec[row] = 1
        output_vec.append(row_vec)
    bpoutput = np.array(output_vec)

    train_in = bpinput[:train_case_num]
    train_out = bpoutput[:train_case_num]
    test_in = bpinput[train_case_num:]
    bpnn = BPNN.BPNN(train_in, train_out)
    bpnn.train(10000)
    test_out = bpnn.predict(test_in)
    test_out = translate_output(test_out)
    test_out_price = (test_out * 2. - 9.) /100. + 1.

    temp = pre_close_price.copy()
    test_case_num = 0
    direction_predict_right_time = 0
    for i in range(len(pre_close_price) - train_case_num):
        pre_close_price[i + train_case_num] = temp[i + train_case_num - 1] * test_out_price[i][0]
        if p_change[i + train_case_num] * (test_out_price[i][0] - 1.) >= 0:
            direction_predict_right_time += 1
        test_case_num += 1

    rp_list.append(direction_predict_right_time/ test_case_num)
    #print('right precentage: ',direction_predict_right_time/ test_case_num)
    #plt.plot(close_price, color='green')
    #plt.plot(pre_close_price, color='red')
    #plt.savefig('E:/stock_data/plot_predict/'+file_name[:-3]+'png')
    #plt.cla()

train_predict_stocks('E:/stock_data/stocks_for_bpnn/', 'bpnn_000012_com.csv', 400)
for bundle in os.walk('E:/stock_data/stocks_for_bpnn/'):
    files = bundle[2]
    i = 0
    for file in files:
        i += 1
        train_predict_stocks(bundle[0], file, 400)
        print('%d files finished.'%i)
plt.plot(rp_list)
plt.show()
plt.savefig('E:/stock_data/r.png')

print(np.mean(rp_list))
print(np.amax(rp_list))
print(np.amin(rp_list))
print(np.std(rp_list))

1 files finished.
2 files finished.
3 files finished.
4 files finished.
5 files finished.
6 files finished.
7 files finished.
8 files finished.
9 files finished.
10 files finished.
11 files finished.
12 files finished.
13 files finished.
14 files finished.
15 files finished.
16 files finished.
17 files finished.
18 files finished.
19 files finished.
20 files finished.
21 files finished.
22 files finished.
23 files finished.
24 files finished.
25 files finished.
26 files finished.
27 files finished.
28 files finished.
29 files finished.
30 files finished.
31 files finished.
32 files finished.
33 files finished.
34 files finished.
35 files finished.
36 files finished.
37 files finished.
38 files finished.
39 files finished.
40 files finished.
41 files finished.
42 files finished.
43 files finished.
44 files finished.
45 files finished.
46 files finished.
47 files finished.
48 files finished.
49 files finished.
50 files finished.
51 files finished.
52 files finished.
53 files finished.
54

In [10]:
import matplotlib.pyplot as plt
import BPNN
import pandas as pd
import os


def normalize_output(df):
    out = []
    for i in range(len(df)):
        pchange_n = int((df.iloc[i]+ 10.) / 2.)
        if pchange_n == 10:
            pchange_n = 9
        elif pchange_n == -1:
            pchange_n = 0
        out += [pchange_n]
    return out


def gen_stock_data_for_bpnn(dir, newdir):
    df = pd.read_csv(dir)
    
    #reverse date
    for i in range(int(len(df)/ 2) - 1):
        df.iloc[i], df.iloc[len(df) - i - 1] = df.iloc[len(df) - i - 1], df.iloc[i]
    
    df['normalized_p_change'] = normalize_output(df['p_change'])
    pcdf = df['p_change'][:-1].tolist()
    pcdf = [0] + pcdf
    df['yesterday_p_change'] = pcdf
    
    closedf = df['close'][:-1].tolist()
    closedf = [0] + closedf
    df['yesterday_close'] = closedf
    
    highdf = df['high'][:-1].tolist()
    highdf = [0] + highdf
    df['yesterday_high'] = highdf
    
    lowdf = df['low'][:-1].tolist()
    lowdf = [0] + lowdf
    df['yesterday_low'] = lowdf
    
    ma5df = df['ma5'][:-1].tolist()
    ma5df = [0] + ma5df
    df['yesterday_ma5'] = ma5df
    
    ma20df = df['ma20'][:-1].tolist()
    ma20df = [0] + ma20df
    df['yesterday_ma20'] = ma20df
    
    todf = df['turnover'][:-1].tolist()
    todf = [0] + todf
    df['yesterday_turnover'] = todf
    
    df = df.ix[20:]
    
    df.to_csv(newdir)
    

for bundle in os.walk('E:/stock_data/stocks_com/'):
    files = bundle[2]
    i = 0
    for file in files:
        i += 1
        fullpath = os.path.join(bundle[0], file)
        newdir = 'E:/stock_data/stocks_for_bpnn/' + 'bpnn_' + file
        #gen_stock_data_for_bpnn(fullpath, newdir)
        print('%d files finished.'%i)

1 files finished.
2 files finished.
3 files finished.
4 files finished.
5 files finished.
6 files finished.
7 files finished.
8 files finished.
9 files finished.
10 files finished.
11 files finished.
12 files finished.
13 files finished.
14 files finished.
15 files finished.
16 files finished.
17 files finished.
18 files finished.
19 files finished.
20 files finished.
21 files finished.
22 files finished.
23 files finished.
24 files finished.
25 files finished.
26 files finished.
27 files finished.
28 files finished.
29 files finished.
30 files finished.
31 files finished.
32 files finished.
33 files finished.
34 files finished.
35 files finished.
36 files finished.
37 files finished.
38 files finished.
39 files finished.
40 files finished.
41 files finished.
42 files finished.
43 files finished.
44 files finished.
45 files finished.
46 files finished.
47 files finished.
48 files finished.
49 files finished.
50 files finished.
51 files finished.
52 files finished.
53 files finished.
54